***Bernouli's Equation***

$$
g (z_1 - z_0) + \frac{P_1 - P_0}{\rho_0} + \frac{v_1^2 - v_0^2}{2} + \left( f_0 \frac{L}{D} + \sum K \right) \frac{v_0^2}{2} = 0
$$

**Constant Stagnation Enthalpy**

$$
h_1 - h_0 + \frac{v_1^2 - v_0^2}{2} = 0
$$

In [1]:
import thermo as th
import thermo.interaction_parameters as ip
import fluids as fl
from math import sqrt, pi
import numpy as np
from scipy.optimize import root

In [2]:
g = 9.81

In [3]:
constants, properties = th.ChemicalConstantsPackage.from_IDs(['methane', 'ethane'])

In [4]:
kijs = ip.IPDB.get_ip_asymmetric_matrix('ChemSep PR', constants.CASs, 'kij')

In [5]:
eos_kwargs = {'Pcs': constants.Pcs, 'Tcs': constants.Tcs, 'omegas': constants.omegas, 'kijs': kijs}

In [6]:
gas = th.CEOSGas(th.PRMIX, eos_kwargs, HeatCapacityGases=properties.HeatCapacityGases)
liquid = th.CEOSLiquid(th.PRMIX, eos_kwargs, HeatCapacityGases=properties.HeatCapacityGases)

In [7]:
flasher = th.FlashVL(constants, properties, liquid=liquid, gas=gas)

In [22]:
T0 = 300.0
P0 = 1e5
zs = [0.5, 0.5]
m = 10.0
L = 100.0
D = 0.1667
K = 4.0
dz = 0.0
state0 = flasher.flash(T=T0, P=P0, zs=zs)

In [13]:
def dp_const_stag_H_segment(state0, m, D, L=0.0, eD=6e-5, K=0.0, dz=0.0):
    def func(vec):
        T1, P1 = vec
        
        # Flash the incoming fluid to the specified downstream conditions T1, P1
        state1 = state0.flasher.flash(T=T1, P=P1, zs=state0.zs)
        
        # Calculate various properties at the inlet and outlet
        v0 = m / state0.rho_mass() * 4 / pi / D / D
        v1 = m / state1.rho_mass() * 4 / pi / D / D
        h0 = state0.H_mass()
        h1 = state1.H_mass()
        rho0 = state0.rho_mass()
        rho1 = state1.rho_mass()
        mu0 = state0.mu()
        
        Re0 = fl.Reynolds(V=v0, D=D, rho=rho0, mu=mu0)
        f0 = fl.friction.friction_factor(Re=Re0, eD=eD)
        
        # Return the values of the root equations for Bernoulli's and the constant stagnation enthalpy equation
        return [g * dz + (P1 - state0.P) / rho0 + (v1 ** 2 - v0 ** 2) / 2 + (f0 * L / D + K) * (v0 ** 2) / 2,
                h1 - h0 + (v1 ** 2 - v0 ** 2) / 2]
    
    return root(func, [state0.T, state0.P])

In [21]:
dp_const_stag_H_segment(state0, m, L, D, K, dz).x

array([  300.        , 99999.99999968])

In [ ]:
timeit(dp_const_stag_H(state0, m, L, D, K, dz))

In [25]:
import pandas as pd
import numpy as np

In [26]:
layout = pd.read_csv('piping.csv')
layout

,name,D,L,eD,K,dz
0,pipe,0.1667,5.0,0.000002,0.0,0.0
1,valve,0.1667,0.0,0.000002,1.3,0.0
2,pipe,0.1667,5.0,0.000002,0.0,5.0


In [23]:
def dp_const_stag_H_layout(state0, m, layout):
    layout['P_in'] = np.ones(3) * np.NAN
    layout['P_out'] = np.ones(3) * np.NAN
    layout['T_in'] = np.ones(3) * np.NAN
    layout['T_out'] = np.ones(3) * np.NAN
    state = state0
    for segment in layout.itertuples():
        segment['P_in'] = state.P
        segment['T_in'] = state.T
        T_out, P_out = dp_const_stag_H_segment(state, m=m, D=segment['D'], L=segment['L'], eD=segment['eD'], K=segment['K'], dz=segment['dz'])
        segment['P_out'] = P_out
        segment['T_out'] = T_out
        state = state.flasher.flash(P=P_out, T=T_out, zs=state.zs)

In [27]:
dp_const_stag_H_layout(state0, m, layout)

TypeError: 'Pandas' object does not support item assignment